# Exploring key predictors of CS enrollment in Washington state

We will be using the data table at https://github.com/lgtanimoto/WAStateCSEd/blob/main/2022/2022_school_pt.csv created from the 2021-22 CSEd Summary Report published by OSPI which shows for each high school, how many students, how many student enrollments, how many male students, how many male student enrollments, how many female students, how many female student enrollments, how many gender X, how many gender X enrollments, and does similar info for race/ethnicity, FRL status, ELL status, disability status.   Cross-sectional data (e.g. how many black female enrollments) is not available. With this data we will generate 300,000 random student records and then perform PCA analysis to discover key predictors of CS enrollment in Washington state.

First, we need to import the libraries we will need to explore and analyze this data.

In [275]:
import pandas as pd
import numpy as np
# import seaborn as sns
import sklearn as sk
# import ctgan as ct
# import sdv
#from sdv.single_table import CTGANSynthesizer
#from sdv.datasets.local import load_csvs
#from ctgan import CTGAN
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics

Then we will upload the data that we will be using to generate random student records.

In [276]:
ospi_data = pd.read_csv('2022_school_pt.csv')

Let's take a look at the first few records from the csv file to better understand the school records we are working with.

In [277]:
ospi_data.head()

,Unnamed: 0,DistrictCode,SchoolCode,AllStudents,C_AllStudents,G_Female,GC_Female,G_Male,GC_Male,G_GenderX,...,D_NoDisability,DC_NoDisability,A_9,AC_9,A_10,AC_10,A_11,AC_11,A_12,AC_12
0,0,1109,3075,28,3,9,0,19,3,0,...,26,3,7,0,4,0,6,1,11,2
1,1,1147,3015,1281,169,635,60,646,109,0,...,1086,140,382,51,328,44,321,29,250,45
2,2,1158,2903,60,22,30,12,29,10,1,...,57,21,18,12,15,2,14,2,13,6
3,3,1160,2132,113,24,57,13,56,11,0,...,107,20,32,7,35,5,25,4,21,8
4,4,2250,1617,167,0,89,0,78,0,0,...,142,0,16,0,51,0,41,0,59,0


In [278]:
len(ospi_data.index)

730

We will continue to group students by schools for future analysis and current analysis--currently we can see how the school affects enrollment as well as enrollment probabilities by feature (does a certain feature's probability change depending on the school they are in?) CREATE FUNCTION TO DO ALL OF THE BELOW FOR EACH ROW OF ABOVE DF SO PROBS ARE UNIQUE TO EACH SCHOOL

Next we need to make sure the data is clean and suitable for manipulation by searching for missing values. We will handle normalization later, in different ways, depending on the method for synthetic data generation.

In [279]:
num_missing_val = ospi_data.isnull().sum()
num_missing_val.sum()

0

We see that there are no missing values so now we will move on to generating 300,000 random student records with the help of numpy.

To do this, we need to gather the probabilities implied by the given dataset. These probabilities need to be calculated from each of the columns--comparing the total number of students belonging to each feature category to the number that enrolled in a CS course. We also need to find probabilities for which category a single student will belong to.

In [280]:
ospi_data.columns

Index(['Unnamed: 0', 'DistrictCode', 'SchoolCode', 'AllStudents',
       'C_AllStudents', 'G_Female', 'GC_Female', 'G_Male', 'GC_Male',
       'G_GenderX', 'GC_GenderX', 'R_Native', 'RC_Native', 'R_Asian',
       'RC_Asian', 'R_Black', 'RC_Black', 'R_Hisp_Lat', 'RC_Hisp_Lat', 'R_HPI',
       'RC_HPI', 'R_NA', 'RC_NA', 'R_TwoOrMore', 'RC_TwoOrMore', 'R_White',
       'RC_White', 'L_ELL', 'LC_ELL', 'L_NoELL', 'LC_NoELL', 'I_LowIncome',
       'IC_LowIncome', 'I_NoLowIncome', 'IC_NOLowIncome', 'D_Disability',
       'DC_Disability', 'D_NoDisability', 'DC_NoDisability', 'A_9', 'AC_9',
       'A_10', 'AC_10', 'A_11', 'AC_11', 'A_12', 'AC_12'],
      dtype='object')

An interesting consideration to begin with is looking at the ratio of all students who enroll in CS:

In [281]:
sum(ospi_data.C_AllStudents) / sum(ospi_data.AllStudents)

0.07654341235652434

This shows that about 7.65% of all students in this dataset enrolled in CS. 

Now, let's go through each of the columns to assign probabilities to every category of student.

In [282]:
Prob_GC_Female = sum(ospi_data.GC_Female) / sum(ospi_data.G_Female)
Prob_GC_Male = sum(ospi_data.GC_Male) / sum(ospi_data.G_Male)
Prob_GC_GenderX = sum(ospi_data.GC_GenderX) / sum(ospi_data.G_GenderX)
Prob_RC_Native = sum(ospi_data.RC_Native) / sum(ospi_data.R_Native)
Prob_RC_Asian = sum(ospi_data.RC_Asian) / sum(ospi_data.R_Asian)
Prob_RC_Black = sum(ospi_data.RC_Black) / sum(ospi_data.R_Black)
Prob_RC_Hisp_Lat = sum(ospi_data.RC_Hisp_Lat) / sum(ospi_data.R_Hisp_Lat)
Prob_RC_HPI = sum(ospi_data.RC_HPI) / sum(ospi_data.R_HPI)
Prob_RC_NA = sum(ospi_data.RC_NA) / sum(ospi_data.R_NA)
Prob_RC_TwoOrMore = sum(ospi_data.RC_TwoOrMore) / sum(ospi_data.R_TwoOrMore)
Prob_RC_White = sum(ospi_data.RC_White) / sum(ospi_data.R_White)
Prob_LC_ELL = sum(ospi_data.LC_ELL) / sum(ospi_data.L_ELL)
Prob_LC_NoELL = sum(ospi_data.LC_NoELL) / sum(ospi_data.L_NoELL)
Prob_IC_LowIncome = sum(ospi_data.IC_LowIncome) / sum(ospi_data.I_LowIncome)
Prob_IC_NoLowIncome = sum(ospi_data.IC_NOLowIncome) / sum(ospi_data.I_NoLowIncome)
Prob_DC_Disability = sum(ospi_data.DC_Disability) / sum(ospi_data.D_Disability)
Prob_DC_NoDisability = sum(ospi_data.DC_NoDisability) / sum(ospi_data.D_NoDisability)
Prob_AC_9 = sum(ospi_data.AC_9) / sum(ospi_data.A_9)
Prob_AC_10 = sum(ospi_data.AC_10) / sum(ospi_data.A_10)
Prob_AC_11 = sum(ospi_data.AC_11) / sum(ospi_data.A_11)
Prob_AC_12 = sum(ospi_data.AC_12) / sum(ospi_data.A_12)

There's more work to do but let's pause and notice that we can now easily compare probabilities, like for gender:

In [283]:
print(Prob_GC_Female)
print(Prob_GC_Male)
print(Prob_GC_GenderX)

0.04510418114369407
0.10592888529774107
0.07277526395173455


These numbers suggest that there's more than double the probability that a male student will enroll in a CS course versus a female student. Also interesting to note is that a student identifying as GenderX has a higher probability of enrolling than a female student. However, this probability may not be completely realistic because there are far fewer students who identify as GenderX, compared to male or female.

In [284]:
print(sum(ospi_data.G_Female))
print(sum(ospi_data.G_Male))
print(sum(ospi_data.G_GenderX))

187078
200493
2652


Next we will find probabilities for which category a student will belong to.

In [285]:
Prob_G_Female = sum(ospi_data.G_Female) / sum(ospi_data.AllStudents)
Prob_G_Male = sum(ospi_data.G_Male) / sum(ospi_data.AllStudents)
Prob_G_GenderX = sum(ospi_data.G_GenderX) / sum(ospi_data.AllStudents)
Prob_R_Native = sum(ospi_data.R_Native) / sum(ospi_data.AllStudents)
Prob_R_Asian = sum(ospi_data.R_Asian) / sum(ospi_data.AllStudents)
Prob_R_Black = sum(ospi_data.R_Black) / sum(ospi_data.AllStudents)
Prob_R_Hisp_Lat = sum(ospi_data.R_Hisp_Lat) / sum(ospi_data.AllStudents)
Prob_R_HPI = sum(ospi_data.R_HPI) / sum(ospi_data.AllStudents)
Prob_R_NA = sum(ospi_data.R_NA) / sum(ospi_data.AllStudents)
Prob_R_TwoOrMore = sum(ospi_data.R_TwoOrMore) / sum(ospi_data.AllStudents)
Prob_R_White = sum(ospi_data.R_White) / sum(ospi_data.AllStudents)
Prob_L_ELL = sum(ospi_data.L_ELL) / sum(ospi_data.AllStudents)
Prob_I_LowIncome = sum(ospi_data.I_LowIncome) / sum(ospi_data.AllStudents)
Prob_I_NoLowIncome = sum(ospi_data.I_NoLowIncome) / sum(ospi_data.AllStudents)
Prob_D_Disability = sum(ospi_data.D_Disability) / sum(ospi_data.AllStudents)
Prob_D_NoDisability = sum(ospi_data.D_NoDisability) / sum(ospi_data.AllStudents)
Prob_A_9 = sum(ospi_data.A_9) / sum(ospi_data.AllStudents)
Prob_A_10 = sum(ospi_data.A_10) / sum(ospi_data.AllStudents)
Prob_A_11 = sum(ospi_data.A_11) / sum(ospi_data.AllStudents)
Prob_A_12 = sum(ospi_data.A_12) / sum(ospi_data.AllStudents)

Now, we can use these probabilities to begin creating a synthetic dataset of student records. Let's start with gender.

In [286]:
rng = np.random.default_rng()

# How many student records we want to generate

num_students = 300000

# Load and normalize probabilities
# Gender

g_probs = np.array([Prob_G_Female, Prob_G_Male, Prob_G_GenderX])
g_probs /= np.sum(g_probs)

# Race

r_probs = np.array([Prob_R_Native, Prob_R_Asian, Prob_R_Black, Prob_R_Hisp_Lat, Prob_R_HPI, Prob_R_NA, Prob_R_TwoOrMore, Prob_R_White])
r_probs /= np.sum(r_probs)

# ELL

e_probs = np.array([1-Prob_L_ELL, Prob_L_ELL])

# Income

i_probs = np.array([Prob_I_NoLowIncome, Prob_I_LowIncome])

# Disability

d_probs = np.array([Prob_D_NoDisability, Prob_D_Disability])

# Grade

a_probs = np.array([Prob_A_9, Prob_A_10, Prob_A_11, Prob_A_12])
a_probs /= np.sum(a_probs)

# Generate values for categorical attributes

rng = np.random.default_rng()
gender = rng.choice(['female', 'male', 'x'], size = num_students, p = g_probs)
race = rng.choice(['Native', 'Asian', 'Black', 'Hisp_Lat', 'HPI', 'NA', 'TwoOrMore', 'White'], size = num_students, p = r_probs)
ell = rng.choice([0, 1], size = num_students, p = e_probs)
income = rng.choice([0, 1], size = num_students, p = i_probs)
disability = rng.choice([0, 1], size = num_students, p = d_probs)
grade = rng.choice(['9', '10', '11', '12'], size = num_students, p = a_probs)

# Print the first few synthetic data points to ensure that this is working as expected
print(gender[:10])
print(race[:10])
print(ell[:10])
print(income[:10])
print(disability[:10])
print(grade[:10])

['female' 'male' 'female' 'male' 'male' 'male' 'male' 'female' 'female'
 'female']
['Hisp_Lat' 'Hisp_Lat' 'White' 'Hisp_Lat' 'White' 'White' 'Hisp_Lat'
 'White' 'Asian' 'White']
[0 0 0 0 0 0 0 0 0 0]
[1 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0]
['12' '11' '9' '9' '11' '10' '10' '9' '9' '11']


Let's create a dataframe to hold these new records and add what we've generated as columns.

In [287]:
synth_students = pd.DataFrame()
synth_students.insert(0, "Gender", gender, True)
synth_students.insert(1, "Race", race, True)
synth_students.insert(2, "ELL", ell, True)
synth_students.insert(3, "Low Income", income, True)
synth_students.insert(4, "Disability", disability, True)
synth_students.insert(5, "Grade", grade, True)

After that we can take a look at some of our student records.

In [288]:
synth_students.head()

,Gender,Race,ELL,Low Income,Disability,Grade
0,female,Hisp_Lat,0,1,0,12
1,male,Hisp_Lat,0,0,0,11
2,female,White,0,0,0,9
3,male,Hisp_Lat,0,1,0,9
4,male,White,0,0,0,11


In [289]:
synth_students.tail()

,Gender,Race,ELL,Low Income,Disability,Grade
299995,female,White,0,0,0,12
299996,male,Hisp_Lat,0,0,0,12
299997,male,Asian,0,1,0,10
299998,female,Hisp_Lat,0,1,0,12
299999,female,Hisp_Lat,0,1,0,9


One thing is missing from this synthetic data--whether or not each synthesized student is enrolled in CS. Creating this column will be a little more complex than the previous because we need to use the probabilities of the column values of each row to calculate whether or not the synthesized student is enrolled in CS.

In order to do this, let's convert all of those synthesized student column values into probabilities that they will be enrolled in CS. E.g. if a column value is "Hisp_Lat" then replace it with the variable Prob_RC_Hisp_Lat that we created earlier.

In [290]:
synth_students.replace("Hisp_Lat", Prob_RC_Hisp_Lat)

,Gender,Race,ELL,Low Income,Disability,Grade
0,female,0.066245,0,1,0,12
1,male,0.066245,0,0,0,11
2,female,White,0,0,0,9
3,male,0.066245,0,1,0,9
4,male,White,0,0,0,11
...,...,...,...,...,...,...
299995,female,White,0,0,0,12
299996,male,0.066245,0,0,0,12
299997,male,Asian,0,1,0,10
299998,female,0.066245,0,1,0,12


Now, we'll do this for each variable.

In [291]:
synth_students.replace("Hisp_Lat", Prob_RC_Hisp_Lat)

,Gender,Race,ELL,Low Income,Disability,Grade
0,female,0.066245,0,1,0,12
1,male,0.066245,0,0,0,11
2,female,White,0,0,0,9
3,male,0.066245,0,1,0,9
4,male,White,0,0,0,11
...,...,...,...,...,...,...
299995,female,White,0,0,0,12
299996,male,0.066245,0,0,0,12
299997,male,Asian,0,1,0,10
299998,female,0.066245,0,1,0,12


But it's not best practice to manipulate the dataframe we have. Instead we should create a new dataframe. There are a few ways to do this. Today we will create a new dataframe out of each column, replace(1) values with the associated probabilities, and then bring them back together in a dataframe so that we can calculate our final column of whether or not the student is enrolled in CS.

1: We are using the attribute "inplace=True" to actually manipulate the new dataframes we create out of the columns. In the previous example we did not use this attribute so the dataframe was not actually changed. 

In [292]:
ell_df = synth_students[['ELL']].copy()

In [293]:
ell_df.replace(1, Prob_LC_ELL, inplace=True)
ell_df.head()

,ELL
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [294]:
zero_prob = Prob_LC_NoELL
ell_df.replace(0, zero_prob, inplace=True)
ell_df.head()

,ELL
0,0.078064
1,0.078064
2,0.078064
3,0.078064
4,0.078064


In [295]:
gend_df = synth_students[['Gender']].copy()
race_df = synth_students[['Race']].copy()
inc_df = synth_students[['Low Income']].copy()
dis_df = synth_students[['Disability']].copy()
gr_df = synth_students[['Grade']].copy()

In [296]:
gend_df.replace('male', Prob_GC_Male, inplace=True)
gend_df.replace('female', Prob_GC_Female, inplace=True)
gend_df.replace('x', Prob_GC_GenderX, inplace=True)
gend_df.head()

,Gender
0,0.045104
1,0.105929
2,0.045104
3,0.105929
4,0.105929


In [297]:
race_df.replace('Native', Prob_RC_Native, inplace=True)
race_df.replace('Asian', Prob_RC_Asian, inplace=True)
race_df.replace('Black', Prob_RC_Black, inplace=True)
race_df.replace('Hisp_Lat', Prob_RC_Hisp_Lat, inplace=True)
race_df.replace('HPI', Prob_RC_HPI, inplace=True)
race_df.replace('NA', Prob_RC_NA, inplace=True)
race_df.replace('TwoOrMore', Prob_RC_TwoOrMore, inplace=True)
race_df.replace('White', Prob_RC_White, inplace=True)
race_df.head()

,Race
0,0.066245
1,0.066245
2,0.072356
3,0.066245
4,0.072356


In [298]:
inc_df.replace(0, Prob_IC_NoLowIncome, inplace=True)
inc_df.replace(1, Prob_IC_LowIncome, inplace=True)
inc_df.head()

,Low Income
0,0.067853
1,0.083896
2,0.083896
3,0.067853
4,0.083896


In [299]:
dis_df.replace(0, Prob_DC_NoDisability, inplace=True)
dis_df.replace(1, Prob_DC_Disability, inplace=True)
dis_df.head()

,Disability
0,0.079513
1,0.079513
2,0.079513
3,0.079513
4,0.079513


In [300]:
gr_df.replace('9', Prob_AC_9, inplace=True)
gr_df.replace('10', Prob_AC_10, inplace=True)
gr_df.replace('11', Prob_AC_11, inplace=True)
gr_df.replace('12', Prob_AC_12, inplace=True)
gr_df.head()

,Grade
0,0.061673
1,0.061897
2,0.103522
3,0.103522
4,0.061897


After all of the columns have had their values replaced with probabilities we need to bring them all together in a single dataframe to calculate the probability of enrollment in CS based on those proabibilities. 

In [301]:
calc_prob = pd.DataFrame()
calc_prob["Gender"] = gend_df
calc_prob["Race"] = race_df
calc_prob["ELL"] = ell_df
calc_prob["Income"] = inc_df
calc_prob["Disability"] = dis_df
calc_prob["Grade"] = gr_df
calc_prob.head()

,Gender,Race,ELL,Income,Disability,Grade
0,0.045104,0.066245,0.078064,0.067853,0.079513,0.061673
1,0.105929,0.066245,0.078064,0.083896,0.079513,0.061897
2,0.045104,0.072356,0.078064,0.083896,0.079513,0.103522
3,0.105929,0.066245,0.078064,0.067853,0.079513,0.103522
4,0.105929,0.072356,0.078064,0.083896,0.079513,0.061897


To do this we will need a function to calculate the probability for each row.

In [302]:
def calc_cs_prob(row):
    return row['Gender'] * row['Race'] * row['ELL'] * row['Income'] * row['Disability'] * row['Grade']

We will use this function on each row using apply()

In [303]:
calc_prob['CS_Enroll_Prob'] = calc_prob.apply(calc_cs_prob, axis=1)

But this just calculates the probabilities. We need to find out if the synthetic student is enrolled or not. So we need to create a second function to use the probability column.

In [304]:
def is_cs(prob):
    return np.random.choice([0, 1], p=[1 - prob, prob])

calc_prob['CS'] = calc_prob['CS_Enroll_Prob'].apply(is_cs)

calc_prob.head()

,Gender,Race,ELL,Income,Disability,Grade,CS_Enroll_Prob,CS
0,0.045104,0.066245,0.078064,0.067853,0.079513,0.061673,7.761077e-08,0
1,0.105929,0.066245,0.078064,0.083896,0.079513,0.061897,2.261875e-07,0
2,0.045104,0.072356,0.078064,0.083896,0.079513,0.103522,1.759370e-07,0
3,0.105929,0.066245,0.078064,0.067853,0.079513,0.103522,3.059521e-07,0
4,0.105929,0.072356,0.078064,0.083896,0.079513,0.061897,2.470558e-07,0


In [305]:
calc_prob.tail()

,Gender,Race,ELL,Income,Disability,Grade,CS_Enroll_Prob,CS
299995,0.045104,0.072356,0.078064,0.083896,0.079513,0.061673,1.048150e-07,0
299996,0.105929,0.066245,0.078064,0.083896,0.079513,0.061673,2.253692e-07,0
299997,0.105929,0.154843,0.078064,0.067853,0.079513,0.079958,5.523605e-07,0
299998,0.045104,0.066245,0.078064,0.067853,0.079513,0.061673,7.761077e-08,0
299999,0.045104,0.066245,0.078064,0.067853,0.079513,0.103522,1.302734e-07,0


In [306]:
sum(calc_prob['CS'])

0

So, this seems strange. There should be more than two students enrolled in CS? Let's look at the ratio of enrolled students in our sample data.

In [307]:
sum(ospi_data.C_AllStudents) / sum(ospi_data.AllStudents)

0.07654341235652434

Which was how many students?

In [308]:
sum(ospi_data.C_AllStudents)

29869

In [309]:
2/300000

6.666666666666667e-06

In [310]:
calc_prob = calc_prob.drop('CS_Enroll_Prob', axis=1)
calc_prob = calc_prob.drop('CS', axis=1)

If you look back up at the probability functions we created you can see the issue; there is no consideration for dependence. So, we want to rewrite the function but are missing important information for conditional probabilities--we know the probability of whether or not somebody enrolls in CS because of one feature, but not multiple features. No cross-sectional or intersectional data is available. So instead we will calculate CS enrollment two ways and decide which is best for our use case. 

In [311]:
def calc_m1_prob(row):
    return min(row['Gender'], row['Race'], row['ELL'], row['Income'], row['Disability'], row['Grade'])


calc_prob['M1_Prob'] = calc_prob.apply(calc_m1_prob, axis=1)


def is_cs_m1(prob):
    return np.random.choice([0, 1], p=[1 - prob, prob])

calc_prob['CS_M1'] = calc_prob['M1_Prob'].apply(is_cs_m1)
calc_prob.head()

,Gender,Race,ELL,Income,Disability,Grade,M1_Prob,CS_M1
0,0.045104,0.066245,0.078064,0.067853,0.079513,0.061673,0.045104,0
1,0.105929,0.066245,0.078064,0.083896,0.079513,0.061897,0.061897,0
2,0.045104,0.072356,0.078064,0.083896,0.079513,0.103522,0.045104,0
3,0.105929,0.066245,0.078064,0.067853,0.079513,0.103522,0.066245,0
4,0.105929,0.072356,0.078064,0.083896,0.079513,0.061897,0.061897,0


Another possible method would be to use the probability found from all students, not taking features into account.

In [312]:
Prob_All = sum(ospi_data.C_AllStudents) / sum(ospi_data.AllStudents)
a_probs = np.array([1-Prob_All, Prob_All])
CS_M2 = rng.choice([0, 1], size = num_students, p = a_probs)
calc_prob.insert(8, "CS_M2", CS_M2, True)
calc_prob.head()

,Gender,Race,ELL,Income,Disability,Grade,M1_Prob,CS_M1,CS_M2
0,0.045104,0.066245,0.078064,0.067853,0.079513,0.061673,0.045104,0,1
1,0.105929,0.066245,0.078064,0.083896,0.079513,0.061897,0.061897,0,0
2,0.045104,0.072356,0.078064,0.083896,0.079513,0.103522,0.045104,0,0
3,0.105929,0.066245,0.078064,0.067853,0.079513,0.103522,0.066245,0,0
4,0.105929,0.072356,0.078064,0.083896,0.079513,0.061897,0.061897,0,0


In [322]:
def calc_m3_prob(row):
    return (row['Gender'] + row['Race'] + row['ELL'] + row['Income'] + row['Disability'] + row['Grade']) / 6


calc_prob['M3_Prob'] = calc_prob.apply(calc_m3_prob, axis=1)


def is_cs_m3(prob):
    return np.random.choice([0, 1], p=[1 - prob, prob])

calc_prob['CS_M3'] = calc_prob['M3_Prob'].apply(is_cs_m3)
calc_prob.head()

,Gender,Race,ELL,Income,Disability,Grade,M1_Prob,CS_M1,CS_M2,M3_Prob,CS_M3
0,0.045104,0.066245,0.078064,0.067853,0.079513,0.061673,0.045104,0,1,0.066409,0
1,0.105929,0.066245,0.078064,0.083896,0.079513,0.061897,0.061897,0,0,0.079257,0
2,0.045104,0.072356,0.078064,0.083896,0.079513,0.103522,0.045104,0,0,0.077076,0
3,0.105929,0.066245,0.078064,0.067853,0.079513,0.103522,0.066245,0,0,0.083521,0
4,0.105929,0.072356,0.078064,0.083896,0.079513,0.061897,0.061897,0,0,0.080276,0


Let's compare the different percentages of CS enrollment between methods and the sample data.

In [323]:
pct_All_CS_synth1 = f"{sum(calc_prob['CS_M1']) / num_students:%}"
pct_All_CS_synth2 = f"{sum(calc_prob['CS_M2']) / num_students:%}"
pct_All_CS_synth3 = f"{sum(calc_prob['CS_M3']) / num_students:%}"
pct_All_CS_ospi = f"{sum(ospi_data.C_AllStudents) / sum(ospi_data.AllStudents):%}"
print("Method 1:", pct_All_CS_synth1)
print("Method 2:",pct_All_CS_synth2)
print("Method 3:",pct_All_CS_synth3)
print("OSPI Data:",pct_All_CS_ospi)

Method 1: 5.444333%
Method 2: 7.682667%
Method 3: 7.713000%
OSPI Data: 7.654341%


Method 2 yields a more similar percentage of enrollment but doesn't take into account features. Method 3 is closer than Method 1 but not quite as close as Method 2. 

To better understand how to do this we can explore the data further. Three ways we will look deeper into this data:

- Combine the probability dataframe with the binary data frame so we can look at these together and make sense of them within each synthetic student. Picking the minimum or average may not be the answer, maybe some probabilities should be weighted? Is the random generator ensuring we aren't creating bias in the PCA we want to later perform?
- Make up the missing percentages between Method 1 and sample data using another random method.
- Divide students by school. This may result in richer analysis, anyway, so we will pursue this option as well for PCA.

In [324]:
all_synth = pd.concat([synth_students, calc_prob], axis=1)
all_synth.head(25)

,Gender,Race,ELL,Low Income,Disability,Grade,Gender,Race,ELL,Income,Disability,Grade,M1_Prob,CS_M1,CS_M2,M3_Prob,CS_M3
0,female,Hisp_Lat,0,1,0,12,0.045104,0.066245,0.078064,0.067853,0.079513,0.061673,0.045104,0,1,0.066409,0
1,male,Hisp_Lat,0,0,0,11,0.105929,0.066245,0.078064,0.083896,0.079513,0.061897,0.061897,0,0,0.079257,0
2,female,White,0,0,0,9,0.045104,0.072356,0.078064,0.083896,0.079513,0.103522,0.045104,0,0,0.077076,0
3,male,Hisp_Lat,0,1,0,9,0.105929,0.066245,0.078064,0.067853,0.079513,0.103522,0.066245,0,0,0.083521,0
4,male,White,0,0,0,11,0.105929,0.072356,0.078064,0.083896,0.079513,0.061897,0.061897,0,0,0.080276,0
5,male,White,0,0,0,10,0.105929,0.072356,0.078064,0.083896,0.079513,0.079958,0.072356,0,0,0.083286,0
6,male,Hisp_Lat,0,0,1,10,0.105929,0.066245,0.078064,0.083896,0.056698,0.079958,0.056698,0,0,0.078465,0
7,female,White,0,0,0,9,0.045104,0.072356,0.078064,0.083896,0.079513,0.103522,0.045104,0,0,0.077076,0
8,female,Asian,0,0,0,9,0.045104,0.154843,0.078064,0.083896,0.079513,0.103522,0.045104,0,0,0.090824,0
9,female,White,0,0,0,11,0.045104,0.072356,0.078064,0.083896,0.079513,0.061897,0.045104,0,0,0.070139,0


After we've created this last column, we can create our final dataset that will be ready for PCA. One way to prepare this dataset is making all columns binary. This will make analysis simpler (TODO: reword this)

In [325]:
binary_gender = pd.get_dummies(synth_students['Gender'])
binary_gender.head()

,female,male,x
0,1,0,0
1,0,1,0
2,1,0,0
3,0,1,0
4,0,1,0


In [326]:
binary_race = pd.get_dummies(synth_students['Race'])
binary_race.head()

,Asian,Black,HPI,Hisp_Lat,NA,Native,TwoOrMore,White
0,0,0,0,1,0,0,0,0
1,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,1


In [327]:
binary_grade = pd.get_dummies(synth_students['Grade'])
binary_grade.head()

,10,11,12,9
0,0,0,1,0
1,0,1,0,0
2,0,0,0,1
3,0,0,0,1
4,0,1,0,0


Then concatenate these new columns with the existing synth_students dataframe and drop the non-binary ones.

In [330]:
binary_synth = pd.concat([synth_students, binary_gender, binary_race, binary_grade, calc_prob['CS_M1'], calc_prob['CS_M2'], calc_prob['CS_M3']], axis=1)
binary_synth = binary_synth.drop('Gender', axis=1)
binary_synth = binary_synth.drop('Race', axis=1)
binary_synth = binary_synth.drop('Grade', axis=1)

binary_synth.head(25)

,ELL,Low Income,Disability,female,male,x,Asian,Black,HPI,Hisp_Lat,...,Native,TwoOrMore,White,10,11,12,9,CS_M1,CS_M2,CS_M3
0,0,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
1,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,0,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
6,0,0,1,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
7,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
8,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


Now we are concerned with normalizing the generated data to prepare for PCA.